<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/163_Agentic_Patterns_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🔎 What are “patterns” in LangGraph?

In this context, **patterns are reusable orchestration blueprints**.
They aren’t code you must copy exactly, but **design idioms** — ways of structuring agents, nodes, and edges to solve common categories of problems.

Think of them like “design patterns” in software engineering (Factory, Observer, etc.), but here they’re **agentic orchestration patterns**:

* They show **how to break down tasks** into steps or branches.
* They guide **how to connect nodes** (linear, branching, iterative).
* They define **how to handle memory, reflection, or evaluation**.

---

## 🛠 Why are patterns useful?

1. **Reusability**
   You don’t have to reinvent orchestration for every new agent — you can slot your domain-specific tools into a proven workflow.

2. **Clarity**
   Patterns give you mental models:

   * Parallel Processing → fan out and merge results.
   * Reflection → iterate until quality threshold.
   * Tree of Thought → branch and evaluate reasoning paths.

3. **Scalability**
   Complex systems are just compositions of simple patterns.
   By learning patterns, you gain LEGO-like building blocks that can be combined.

4. **Reliability**
   Patterns encode best practices (like retry, review loops, consensus).
   That means fewer dead ends, hallucinations, or brittle flows.

---

✅ **So at a high level:**
Patterns are **structured agent workflows** that balance exploration, feedback, and decision-making. They’re useful because they abstract away *how to orchestrate* so you can focus on *what your agent should do*.



The **parallel processing pattern** is one of the most useful ones to learn in LangGraph because it introduces the idea of **fan-out and fan-in orchestration**. Let’s unpack what’s happening in your `parallel_processing.py` example.

---

## 🔑 What This Pattern Does

* **Fan-out**: At the `START` node, the graph launches *multiple independent subtasks* in parallel:

  * `fetch_trends`
  * `analyze_competitors`
  * `extract_sentiment`

* **Independent execution**: Each of these nodes runs separately, queries the LLM, and writes back to different parts of the shared `MarketResearchState`.

* **Fan-in (merge)**: All three outputs flow into the `summarize` node. This node reads the collected state (`trends`, `competitors`, `sentiment`) and produces a final `summary`.

---

## 🧠 What You Should Focus On

1. **State Design**

   * Notice how `MarketResearchState` defines fields for each subtask (`trends`, `competitors`, `sentiment`, and `summary`).
   * This ensures the outputs don’t collide — each node has a “slot” in memory.

2. **Parallel Edges**

   * Multiple `graph_builder.add_edge(START, ...)` create branches that run independently.
   * They all converge back into `summarize`.

3. **Fan-in Dependency**

   * The `summarize` node isn’t called until *all three feeder nodes have finished*.
   * LangGraph handles that orchestration automatically — you don’t need to manually synchronize.

4. **Pattern Recognition**

   * This is a reusable blueprint: any time you want to **gather multiple perspectives / analyses in parallel and then combine them**, you use this fan-out/fan-in shape.

---

## ✨ Why It’s Useful

* **Efficiency**: You can run multiple LLM calls in parallel rather than sequentially.
* **Clarity**: The graph visually and structurally shows you’re collecting multiple “ingredients” before producing a final result.
* **Extensibility**: You can easily add more branches (e.g. `regulatory_analysis`, `pricing_analysis`) without changing the orchestration logic.
* **Reliability**: If one node fails, you can add error edges or fallbacks without disrupting the whole pattern.

---

## 📌 What to Learn From This Example

* How to **define state fields** to keep parallel outputs organized.
* How to **branch out from START** into multiple subtasks.
* How LangGraph automatically **waits for all upstream nodes** before executing the downstream aggregator.
* That this pattern is essentially a **map-reduce workflow**: map (independent tasks) → reduce (summary node).

---

👉 Would you like me to draw this as an ASCII flowchart (fan-out/fan-in shape) so you can see the structure visually, the same way we did for your earlier orchestrator?


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI


# Define state
class MarketResearchState(TypedDict):
    query: str
    trends: str
    competitors: str
    sentiment: str
    summary: str


llm = ChatOpenAI()


def fetch_trends(state: MarketResearchState):
    response = llm.invoke(f"What are the latest market trends for {state['query']}?")
    return {"trends": response.content}


def analyze_competitors(state: MarketResearchState):
    response = llm.invoke(f"List top competitors in {state['query']} market.")
    return {"competitors": response.content}


def extract_sentiment(state: MarketResearchState):
    response = llm.invoke(f"What do customers feel about products in {state['query']} category?")
    return {"sentiment": response.content}


def summarize(state: MarketResearchState):
    summary_prompt = f"""
    Product Research Summary:
    - Trends: {state.get('trends')}
    - Competitors: {state.get('competitors')}
    - Customer Sentiment: {state.get('sentiment')}
    Provide strategic insights for entering the {state['query']} market.
    """
    response = llm.invoke(summary_prompt)
    return {"summary": response.content}


graph_builder = StateGraph(MarketResearchState)

# Add nodes
graph_builder.add_node("fetch_trends", fetch_trends)
graph_builder.add_node("analyze_competitors", analyze_competitors)
graph_builder.add_node("extract_sentiment", extract_sentiment)
graph_builder.add_node("summarize", summarize)

# TODO: Add edges for parallel execution
graph_builder.add_edge(START,"fetch_trends")
graph_builder.add_edge(START,"analyze_competitors")
graph_builder.add_edge(START,"extract_sentiment")

graph_builder.add_edge("fetch_trends","summarize")
graph_builder.add_edge("analyze_competitors","summarize")
graph_builder.add_edge("extract_sentiment","summarize")

# Compile graph
graph = graph_builder.compile()

# Run it
inputs = {"query": "Smart Water Bottle"}
result = graph.invoke(inputs)

# Output
print("\n=== Final Market Summary ===\n")
print(result["summary"])

There’s actually **more than one reason** to use parallel processing in LangGraph.

---

## ⚡️ 1. Speed (the obvious one)

* Instead of doing:

  ```
  trends → competitors → sentiment → summary
  ```

  (sequentially, where each step waits on the last),
* You do:

  ```
  trends   ┐
  competitors ├──> summary
  sentiment ┘
  ```

  (in parallel, all three run at once).
* That cuts latency roughly down to the longest branch instead of the sum of all branches.

---

## 🧠 2. Independence of Subtasks

* Many subtasks **don’t depend on each other** (e.g., fetching trends doesn’t need competitor analysis).
* Running them sequentially would add artificial coupling.
* Parallelization keeps logic *true to the problem structure*.

---

## 🗂️ 3. Separation of Concerns

* Each branch writes to its own **state slot**.
* That enforces **modularity**: your “trends analysis” can evolve separately from “competitor analysis.”
* This makes maintenance and scaling easier.

---

## 🧪 4. Diversity of Perspectives

* Parallel tasks are often **different perspectives** on the same data.
* By fanning out, you encourage multiple “angles” to be captured before aggregation.
* This is especially useful in agentic patterns like **Tree of Thoughts** or **debate agents**.

---

## 🔄 5. Natural Fit for Orchestration Tools

* LangGraph automatically **waits for all inputs** before moving forward.
* That makes fan-out/fan-in a clean way to ensure your final node has *complete* context.
* It’s essentially a **MapReduce pattern**: map (parallel branches), then reduce (merge).

---

✅ **So yes, speed is the headline advantage, but the *real power* comes from clarity, modularity, and the ability to collect multiple perspectives cleanly.**



Because parallel processing is powerful, but **not always the right choice**. Here are the main times you *wouldn’t* want to use it:

---

## ❌ 1. When Tasks Depend on Each Other

* If **Task B requires Task A’s output**, you can’t parallelize them.
  Example: You must summarize a document *before* you can critique that summary.
* Forcing parallelism here would either fail or produce nonsense.

---

## ❌ 2. When Order Matters

* If tasks need to happen in a strict sequence (like steps in a math proof, or a multi-stage plan), parallelizing breaks the logical flow.
* Think of it as: you can’t “bake the cake” in parallel with “mix the batter.”

---

## ❌ 3. When Resources Are Constrained

* Each branch is usually an LLM call → more **tokens and cost**.
* If you’re on a tight budget, parallelism multiplies expenses.
* Sequential processing may be slower but cheaper.

---

## ❌ 4. When You Want a Single Coherent Narrative

* Multiple branches = multiple perspectives. That’s great for research, but not if you want one **continuous, logically tight chain of thought**.
* Example: writing a story chapter by chapter — if each “chapter agent” runs in parallel, they’ll diverge.

---

## ❌ 5. When Concurrency Overhead Outweighs Gains

* For very lightweight tasks (e.g., string formatting, metadata lookups), the parallel setup adds complexity with little benefit.
* Parallel processing shines when subtasks are heavy or slow.

---

### ✅ Rule of Thumb

Use parallelism when subtasks are:

* Independent,
* Expensive enough to justify parallel calls,
* And meant to be merged at the end.

Avoid it when subtasks are **sequentially dependent, order-sensitive, or cost-sensitive**.





## 🔄 What the Reflection Pattern Is

It’s an **iterative loop** where the agent:

1. **Generates** an initial solution (in your code: Python code for a given problem).
2. **Reviews** that solution, producing feedback and a score.
3. **Improves** the solution using the feedback.
4. **Loops** back to review → improve → review, until:

   * The score is “good enough” (≥9 in this case), or
   * The max number of iterations is reached (3 here).

Finally, it exits and returns the refined output.

This is basically giving the agent a “self-critique + self-improvement” cycle.

---

## 🧠 What You Should Learn Here

### 1. **Self-Improvement Through Feedback**

Instead of a single LLM pass, the agent improves results over multiple cycles.

* First draft may be weak.
* Review step identifies flaws.
* Improvement step fixes them.
* Each loop pushes quality upward.

### 2. **Separation of Roles**

* **Generator** → “Writes code.”
* **Reviewer** → “Critiques code with a score.”
* **Improver** → “Applies changes.”
  Each role has a clear persona (expert dev, senior reviewer, refiner).
  This mirrors real-world workflows (writer → editor → rewriter).

### 3. **Scoring & Stopping Criteria**

The loop doesn’t run forever. It uses:

* **Review score ≥ 9** → stop.
* **Iteration ≥ 3** → stop.
  This teaches you that reflection agents need **objective criteria** to avoid infinite loops.

### 4. **LangGraph’s Power**

Notice how clean the orchestration is:

* Just three nodes (`generate_code`, `review_code`, `improve_code`).
* `Command(goto=...)` decides whether to loop back or end.
* State carries along the current code, feedback, score, and iteration count.

---

## 🧩 Why This Pattern Matters

* **Quality improvement**: Better than one-shot answers.
* **Reusability**: Works for code, essays, plans, images… anything improvable.
* **Transparency**: You can inspect every iteration’s output.
* **Reliability**: Adds checkpoints (review + score) instead of blind generation.

---

✅ **So the key takeaway here is:** Reflection agents let your LLM *simulate an editor/critic loop*, producing more polished outputs with clear stop conditions.


In [ ]:
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import TypedDict
from langgraph.graph import END
from langgraph.types import Command

# Initialize OpenAI model
llm = ChatOpenAI()


# Define Agent State
class CodeState(TypedDict):
    problem_statement: str
    generated_code: str
    review_feedback: str
    refined_code: str
    iteration: int
    review_score: float


# 🟢 Step 1: Generate Initial Code
def generate_code(state: CodeState):
    print("Generating Code")
    prompt = f"""
    Write a clean, efficient, and well-commented Python solution for the following problem:

    {state['problem_statement']}
    """
    response = llm.invoke([SystemMessage(content="You are an expert Python developer."), HumanMessage(content=prompt)])

    state["generated_code"] = response.content
    state["iteration"] = 1
    state["review_score"] = 0
    return Command(goto="review_code", update=state)


# 🟢 Step 2: Review the Generated Code with a Score
def review_code(state: CodeState):
    print("Reviewing Code")
    prompt = f"""
    Review the following Python code for correctness, readability, efficiency, and best practices:

    {state['generated_code']}

    Provide a list of improvements and necessary changes.
    Also, give a **review score** (1-10) for:
    - Correctness
    - Readability
    - Efficiency
    - Maintainability

    Provide the average score out of 10 at the end.
    The last line should contain just the final score as a final_score:score
    """
    response = llm.invoke(
        [SystemMessage(content="You are a senior software engineer reviewing code."), HumanMessage(content=prompt)]
    )

    state["review_feedback"] = response.content

    # Extract the review score from AI feedback (assuming last line is "final_score: 8")
    try:
        lines = response.content.split("\n")
        last_line = lines[-1]
        state["review_score"] = float(last_line.split(":")[-1].strip())
    except:
        print("EXCEPT")
        state["review_score"] = 5  # Default if parsing fails

    return Command(goto="improve_code", update=state)


# 🟢 Step 3: Improve Code Based on Feedback
def improve_code(state: CodeState):
    print("Improving Code")
    print("Review Score ", state["review_score"], "Iteration ", state["iteration"])
    # TODO: Stop Iteration
    if state["review_score"] >= 9 or state["iteration"] >= 3:
        state["refined_code"] = state["generated_code"]
        return Command(goto=END)

    prompt = f"""
    Here is the initial Python code:

    {state['generated_code']}

    And here is the review feedback:

    {state['review_feedback']}

    Apply the suggested improvements and rewrite the code with better efficiency, readability, and correctness.
    """
    response = llm.invoke([SystemMessage(content="You are an AI code refiner."), HumanMessage(content=prompt)])

    state["generated_code"] = response.content
    state["iteration"] += 1
    return Command(goto="review_code", update=state)


# 🔵 Build the LangGraph Workflow
workflow = StateGraph(CodeState)

# Adding Nodes
workflow.add_node("generate_code", generate_code)
workflow.add_node("review_code", review_code)
workflow.add_node("improve_code", improve_code)

# Define Execution Flow
workflow.set_entry_point("generate_code")

# Compile Graph
graph = workflow.compile()

# 🟢 Run Example
input_data = {
    "problem_statement": "Write a function to find the factorial of a number in Python."
}

result = graph.invoke(input_data)

print("🚀 Final Code After Reflection:\n", result["generated_code"])
print("\n🔍 Final Review Feedback:\n", result["review_feedback"])

🎯 One of the biggest joys of working with **LangGraph**: once you learn the core building blocks, every new orchestrator is just a **plug-and-play swap of nodes**.

The recipe always looks the same:

1. **Define the State**

   * A `TypedDict` (or Pydantic model) that holds whatever fields you need for memory, feedback, results, etc.
   * This is the *contract* that your nodes read/write.

2. **Write your Node functions**

   * Each node is just a Python function: takes `state`, returns `{updates}`.
   * Could be a tool call, an LLM prompt, a database query, whatever.
   * If you want reflection, you add nodes like `generate`, `review`, `improve`.

3. **Build the Graph**

   * `builder = StateGraph(State)`
   * `builder.add_node("generate", generate_fn)`
   * `builder.add_node("review", review_fn)`
   * `builder.add_node("improve", improve_fn)`

4. **Wire Edges**

   * Normal flow: `builder.add_edge("generate", "review")`
   * Loops/conditions: `Command(goto="improve")` or `END`.
   * Error/fallbacks: add error edges if you want resilience.

5. **Set entry point + compile**

   * `builder.set_entry_point("generate")`
   * `graph = builder.compile()`

6. **Run**

   * `graph.invoke({"prompt": "..."})`

---

So whether you’re doing:

* Parallel processing (fan-out/fan-in),
* Reflection loops,
* Tree-of-thought branching,
* Or something custom…

👉 the orchestration scaffolding is always the same. Only your **nodes** (tools/logic) change.

---

✅ **Takeaway:** once you’ve internalized `add_node → add_edge → set_entry_point → compile`, you can build entirely new orchestrators by just swapping in different functions. Everything else is boilerplate that stays the same.





## 🌳 What the Tree of Thought (ToT) Pattern Is

This workflow mimics a **branch-and-evaluate reasoning process**:

1. **Generate branches** → multiple candidate strategies (`expansion_options`).
2. **Analyze each branch** → evaluate them individually (`strategy_analysis`).
3. **Select the best branch** → consolidate analyses into a decision (`best_strategy`).

It’s called “Tree of Thought” because instead of one straight line of reasoning, the agent explores *multiple paths in parallel* before pruning down to one.

---

## 🧠 What You Should Learn Here

### 1. **Exploration vs. Exploitation**

* The agent doesn’t just pick the first idea.
* It explores *several possible paths* (like branches of a tree).
* Then it evaluates and prunes back to the strongest.
  This is crucial for tasks where quality depends on considering alternatives.

### 2. **Structured Evaluation**

* Each branch gets a dedicated analysis node (`analyze_strategy`).
* The analysis uses criteria (cost, risk, ROI).
* This enforces **explicit evaluation dimensions** rather than fuzzy intuition.

### 3. **Decision Consolidation**

* A final node (`select_best_strategy`) makes the call.
* This stage integrates prior evaluations into a ranked choice.
* You’re seeing how LangGraph can implement **multi-agent debate or tournament-style reasoning**.

### 4. **LangGraph Orchestration Advantage**

* Even though this looks like a multi-step reasoning pipeline, it’s still just:

  * Define state (with fields for options, analyses, and decision).
  * Add nodes (generate → analyze → select).
  * Wire edges.
  * Compile & run.
* Same scaffold, just a different reasoning *pattern*.

---

## 📈 Why It Matters

Tree of Thought is useful when:

* **Open-ended brainstorming** is required (multiple possible answers).
* **Trade-offs** need to be considered systematically.
* **Decisions** must be justified (with structured analysis).

Examples:

* Strategy selection (as in your demo).
* Product design trade-offs.
* Policy recommendations.
* Even math/logic problems (different solution paths explored in parallel).

---

✅ **Takeaway:** This pattern teaches you how to structure agents to *explore broadly, evaluate deeply, and converge intelligently*. It’s about going beyond single-pass answers to simulate deliberation.



In [ ]:
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import Dict, List

# Initialize OpenAI model
llm = ChatOpenAI()


# Define Agent State
class StrategyState(Dict):
    business_type: str
    expansion_options: List[str]
    strategy_analysis: Dict[str, str]
    best_strategy: str


# 🟢 Step 1: Generate Expansion Strategies
def generate_expansion_options(state: StrategyState) -> StrategyState:
    prompt = f"""
    The company specializes in {state['business_type']}. Suggest three possible expansion strategies:

    1. Entering a new geographical market.
    2. Launching a new product line.
    3. Partnering with an existing brand.

    Provide a brief overview of each strategy.
    """

    response = llm.invoke([SystemMessage(content="You are a business strategist."), HumanMessage(content=prompt)])

    state["expansion_options"] = response.content.split("\n")[:3]  # Extract first three options
    return state


# 🟢 Step 2: Analyze Each Strategy (Breaking Down Into ToT Paths)
def analyze_strategy(state: StrategyState) -> StrategyState:
    strategy_analysis = {}

    for strategy in state["expansion_options"]:
        prompt = f"""
        Analyze the following business expansion strategy:

        {strategy}

        Evaluate it based on:
        - Cost implications
        - Risk factors
        - Potential return on investment (ROI)

        Provide a structured breakdown.
        """
        response = llm.invoke([SystemMessage(content="You are a business analyst."), HumanMessage(content=prompt)])
        strategy_analysis[strategy] = response.content

    state["strategy_analysis"] = strategy_analysis
    return state


# 🟢 Step 3: Choose the Best Strategy (Final Decision)
def select_best_strategy(state: StrategyState) -> StrategyState:
    prompt = f"""
    Given the following business expansion strategies and their analysis:

    {state['strategy_analysis']}

    Rank these strategies based on:
    - Highest ROI
    - Lowest risk
    - Overall feasibility

    Select the **best** strategy and explain why it is the optimal choice.
    """

    response = llm.invoke(
        [SystemMessage(content="You are an expert business strategist."), HumanMessage(content=prompt)])

    state["best_strategy"] = response.content
    return state


# 🔵 Build the LangGraph Workflow
workflow = StateGraph(StrategyState)

# Adding Nodes
workflow.add_node("generate_expansion_options", generate_expansion_options)
workflow.add_node("analyze_strategy", analyze_strategy)
workflow.add_node("select_best_strategy", select_best_strategy)

# Define Execution Flow
workflow.set_entry_point("generate_expansion_options")
workflow.add_edge("generate_expansion_options", "analyze_strategy")
workflow.add_edge("analyze_strategy", "select_best_strategy")

# Compile Graph
graph = workflow.compile()

# 🟢 Run Example
input_data = {
    "business_type": "AI-based EdTech Startup"
}

result = graph.invoke(input_data)

print("🚀 AI-Generated Expansion Strategies:\n", result["expansion_options"])
print("\n🔍 Strategy Analysis:\n", result["strategy_analysis"])
print("\n🏆 Best Strategy Selected:\n", result["best_strategy"])

What you’re seeing here is a **mix of progressive and “parallel-like” steps**:

---

### 1. Progressive flow

The graph is wired strictly as a linear chain:

```
generate_expansion_options → analyze_strategy → select_best_strategy
```

That means you **can’t analyze until you’ve generated**, and you **can’t select until you’ve analyzed**. So orchestration-wise, it’s progressive (step-by-step).

---

### 2. Parallel work *inside a node*

The **illusion of parallelism** happens in `analyze_strategy`.
That node loops over *all* generated options:

```python
for strategy in state["expansion_options"]:
    response = llm.invoke(...)
    strategy_analysis[strategy] = response.content
```

So while the graph shows it as a single node, conceptually it’s **fan-out and fan-in collapsed into one step**:

* *Fan-out*: each option is analyzed independently.
* *Fan-in*: results are merged back into `strategy_analysis`.

If you wanted *true LangGraph parallelism* (like in your market research example), you’d break this into multiple nodes, one per strategy, and merge them. But here it’s handled by Python iteration inside one node.

---

### 3. Why it’s useful

This design pattern teaches you that:

* Some patterns (like reflection or ToT) mix **progressive stages** (generate → evaluate → decide) with **internal branching** (looping or parallel analysis of multiple ideas).
* You don’t always need to blow out the graph with explicit parallel edges if a loop works fine.
* The trade-off: explicit graph parallelism gives you concurrency + orchestration transparency, while loops-in-a-node give you simplicity at the cost of less visibility.

---

✅ **So the takeaway**:
This ToT example is progressive at the orchestration level, but it **packs a parallel “branch exploration” inside one node**. That’s why it feels like a hybrid.


